In [ ]:
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd
from dotenv import load_dotenv
import os
import time
import base64
import hmac
from hashlib import sha1 as sha
import json
import requests

In [ ]:
load_dotenv("local.env")


host = "http://api.velocityweather.com/v1"
access_key = os.getenv("BARON_KEY")
if access_key is None:
    raise ValueError("Missing BARON_KEY in local.env file")
access_key_secret = os.getenv("BARON_SECRET")
if access_key_secret is None:
    raise ValueError("Missing BARON_SECRET in local.env file")


def sign_old(string_to_sign, secret):
    """ Returns the signature for string_to_sign
    """
    signature_bytes = base64.urlsafe_b64encode(
        hmac.new(secret.encode('utf-8'), string_to_sign.encode('utf-8'), sha).digest())
    signature = signature_bytes.decode('utf-8')

    return signature.replace('/', '_').replace('+', '-').rstrip('=')


def sign(string_to_sign, secret):
    hmac_sha1 = hmac.new(
        # Convert secret to bytes using UTF-8
        secret.encode('utf-8'),
        # Convert input string to bytes using UTF-8
        string_to_sign.encode('utf-8'),
        sha                        # Use SHA1 hash algorithm
    )

    # Get the binary digest
    hmac_digest = hmac_sha1.digest()

    # Encode to base64
    base64_encoded = base64.b64encode(hmac_digest).decode('utf-8')

    # Replace characters to make URL safe
    signature = base64_encoded.replace('/', '_').replace('+', '-')

    return signature


def sign_request(url, key, secret):
    """ Returns signed url
    """

    ts = str(int(time.time()))
    sig = sign(key + ":" + ts, secret)
    q = '?' if url.find("?") == -1 else '&'
    url += "%ssig=%s&ts=%s" % (q, sig, ts)
    return url

In [ ]:
def point_query_region(product, product_config, region_bounds):
    meta_url = f'{host}/{access_key}/meta/tiles/product-instances/{product}/{product_config}?page_size=1'
    meta_url = sign_request(meta_url, access_key, access_key_secret)

    try:
        response = requests.get(meta_url)
    except requests.exceptions.RequestException as e:
        print('Request failed:', e)
        return
    assert response.status_code == 200

    # Decode the product instance response and get the most recent product instance time,
    # to be used in the WMS image request.
    content = json.loads(response.text)
    product_instance_time = content[0]['time']

    w_lon = region_bounds[0]
    n_lat = region_bounds[1]
    e_lon = region_bounds[2]
    s_lat = region_bounds[3]
    query_region = f'w_lon={w_lon}&n_lat={n_lat}&e_lon={e_lon}&s_lat={s_lat}'

    url = f'{host}/{access_key}/point/region/{product}/{product_config}/{product_instance_time}.json?{query_region}'
    url = sign_request(url, access_key, access_key_secret)

    headers = {
        'Accept': 'application/json',
        'Accept-Encoding': 'gzip'
    }

    try:
        response = requests.get(url, headers=headers)
        print(f'Response status code: {response.status_code}')
        # print(f'Response headers: {response.headers}')
        # print(f'Response content: {response.content}')
    except requests.exceptions.RequestException as e:
        print('Request failed:', e)
        return
    assert response.status_code == 200

    content = json.loads(response.text)

    return content

In [ ]:
tx_panhandle_bound_box = [-103.05, 34.65, -99.99, 36.53]
tx_panhandle_bound_box_polygon = Polygon([
    (tx_panhandle_bound_box[0], tx_panhandle_bound_box[1]),
    (tx_panhandle_bound_box[2], tx_panhandle_bound_box[1]),
    (tx_panhandle_bound_box[2], tx_panhandle_bound_box[3]),
    (tx_panhandle_bound_box[0], tx_panhandle_bound_box[3]),
])

# Create a GeoDataFrame from the bounding box polygon
pan_gdf = gpd.GeoDataFrame(
    {'geometry': [tx_panhandle_bound_box_polygon]},
    crs="EPSG:4326"
)

In [ ]:
def frange(start, stop, step):
    """Generate a range of floating point numbers."""
    while start < stop:
        yield round(start, 6)
        start += step

In [ ]:
texas_gdf = gpd.read_file("texas.geojson")
texas_gdf

In [ ]:
# Texas Panhandle area
min_lat, max_lat = 32.65, 36.66
min_lon, max_lon = -103.05, -99.99

# bounding box for the entire state of Texas
# min_lat, max_lat = 25.8, 36.8
# min_lon, max_lon = -106.6, -93.5
lat_step = lon_step = 1.0

# change the product the one you want the output for
baron_product = 'C39-0x04A6-0'
baron_product_config = 'Standard-Geodetic'

cells_gdf = None
points_gdf = None
cell_count = 1

for lat in frange(min_lat, max_lat - lat_step + 0.01, lat_step):
    for lon in frange(min_lon, max_lon - lon_step + 0.01, lon_step):
        print(f"Box: N:{lat+1}, S:{lat}, E:{lon+1}, W:{lon}")
        temp_bound_box = [lon, lat + lat_step, lon + lon_step, lat ]

        cell_polygon = Polygon([
            (lon, lat),
            (lon + lon_step, lat),
            (lon + lon_step, lat + lat_step),
            (lon, lat + lat_step),
        ])

        # Check if the cell polygon is within the Texas polygon
        if not cell_polygon.intersects(texas_gdf.geometry[0]):
            print(f"Box: N:{lat+1}, S:{lat}, E:{lon+1}, W:{lon} is outside of Texas")
            continue

        # call Baron API
        response = point_query_region(baron_product, baron_product_config, temp_bound_box)

        if response is None:
            print("No response")
            continue
        
        # write the response to a file
        with open(f"response_{baron_product}_{baron_product_config}_cell{cell_count}.json", "w") as f:
            json.dump(response, f, indent=4)

    cell_count += 1
    # get data for just one cell for now
    if cell_count > 1:
        break
      